## Regressor-VAE - Trained for Data Generation
### Using VolFrac as a constraint.

## <u>__31 December</u>:__ After reading the beta-VAE paper again, and watching part of DeepFindr's "GNN Project #4.2 - GVAE Training and Adjacency reconstruction" video, I saw the chart between z dimension and beta value... so I'm going to try an encoded dimension to... TBD... and see if $\beta$ = 1.0 does any better... 
#### Encoded (Z) dimensions and Betas tested:
##### (128) $\beta$ = 1.0, 0.1, 0.01...
##### (200) $\beta$ = 0.1, 0.01, 0.001... 0.0001 gets results...
##### (256) $\beta$ = 0.01
### __Conclusion...__ I think the z dimension doesn't matter that much in this case... $\beta$ needs to be small...

## <u>__24 December</u>:__ FAIL... You didn't train on 0.1-0.5 volFrac... you forgot to change the directory of the data split... you __DUMMY__
### ... soooo... trying again.... you fool. ATTENTION TO DETAIL (saves lives lol)

## <u>__23 December</u>:__ Trying out Latin Hypercube sampling with the dataset of only 0.1-0.5 volume fraction... trained the model in this notebook...

### <u>__14 December__</u>: Training a new one without KLD Annealing, want to see if the generated data is all that different

### <u>__13 December__</u>: Continuing training without KLD annealing, another 100 epochs or so... I want the val_loss to be around 0.02, ... 0.05?... comparable to previous models...?

### <u>__12 December__</u>: Trained using KLD Annealing, behavior exactly what I expected... Val_Loss, and specifically Reconstruction Loss, is stupid high if beta is > 0.0001


In [ ]:
import os
import workflow_utils

from workflow_utils.FileDirectory import Directory

dirs = Directory(rootpath = '/data/tigusa1/MLTO_UCAH/MLTO_2023/')

In [ ]:
enc_filt_bl = 32
dec_filt_bl = 32
enc_dim = 16
# enc_dim = 256 # !!!!!!!!!!!!!! <><><>< <<<----- 31 December

outer_layer_dim=64

input_shape = (1, 64,64,64)
input_dim = input_shape[-1]

nbpath = os.path.join(dirs._10_VAE_regressor, 'rVAE_archA')
cp_dir = os.path.join(nbpath, 'model_CPs')

# 1 param - Volume Fraction
params = ['volFrac']
Y_file_suffix= 'VF' 
parlen = len(params)
archver = 'A'
archver = f'DataGen_VolFrac_rVAE_Arch{archver}_enc{enc_dim}_{input_dim}in_{outer_layer_dim}outerLyr_sigmoid_large_Z_larger_beta'

In [ ]:
print(archver)

In [ ]:
import pandas as pd
import numpy as np
import json
import glob
import os
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch

import torch.utils.data as data
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim
import torchsummary

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

from workflow_utils.DataPrep import Plot_Array

In [ ]:
# torch.cuda.set_device(0)

In [ ]:
# dir(torch.cuda)

In [ ]:
from workflow_utils.DatasetCreator import DatasetCreator as DC

source_data_path = dirs.source_data_path

In [ ]:
# """
# This cell sets up the DatasetCreator class. In the version of this package as delivered, the X data does not need to be reprocessed.
# The use of testset_seed and having seed=10 ensures that the dataset is split in a replicable way.
# """

split_dir = dirs.split_seed_68_forVAE_nonaugm
csvname = 'data_3phys_w_dyn_topos_scaled.csv'
volfrac_range = (0.01, 0.5)
seed = 68



dataobj = DC(nntype='cnn', data_source_directory = source_data_path, created_directory=split_dir, yfn = csvname)
dataobj.LoadSpreadsheet()
dataobj.FilterVolFrac(volfrac_range = volfrac_range, bound='none')
dataobj.TrainTestSplit(testset_seed=seed, tf_te_subset=['lattice', 'topopt'], train_size=0.9, omit_tf=False, subset_tf=False, lens=[2,2])

In [ ]:
"""
Calling ProcessXYcnn with Ys_only set to True: this will produce Y arrays in train/validation/test split without re-splitting the X data.
This flag saves time by enabling use of the same split of data while selecting different parameter(s) as the regression target(s).s

If it is desired to create a new split, use the code notebook "Split_and_Assemble_Dataset/split_dataset.ipynb", which will enable saving
the assembled X data as separate arrays ready for loading.

"""

dataobj.ProcessXYcnn(params=params, Ys_only=True, skip_Ys=True, Y_file_suffix=Y_file_suffix, translate=False, translate_by=(0.25,0.5))

In [ ]:
# To load in the compressed data - used in other notebooks, included here for reference.
# Xtr =   np.load(f'{split_dir}Xtr_compressed.npy')
# Xval =  np.load(f'{split_dir}Xval_compressed.npy')
# Xte =   np.load(f'{split_dir}Xte_compressed.npy')

Xtr =   np.load(f'{split_dir}Xtr_fullres_unpad.npy')
Xval =  np.load(f'{split_dir}Xval_fullres_unpad.npy')
Xte =   np.load(f'{split_dir}Xte_fullres_unpad.npy')


In [ ]:
Ytr  = np.load(f'{split_dir}Ytr{Y_file_suffix}.npy')
Yval = np.load(f'{split_dir}Yval{Y_file_suffix}.npy')
Yte  = np.load(f'{split_dir}Yte{Y_file_suffix}.npy')

In [ ]:
print(Xtr.shape, Ytr.shape)
print(Xval.shape, Yval.shape)
print(Xte.shape, Yte.shape)

In [ ]:
# This dataset creator class modified for Regression VAE

class RVAEDataset(data.Dataset):


    def __init__(self, Xarray, Yarray):
        """
        Inputs:
            size - Number of data points we want to generate
            std - Standard deviation of the noise (see generate_continuous_xor function)
        """
        super().__init__()
        self.Xarray=Xarray
        self.Yarray=Yarray
        self.size = Xarray.shape[0]
        self.setup_dataset(self.Xarray, self.Yarray)

    # def generate_continuous_xor(self):
    def setup_dataset(self, Xarray, Yarray):
        data = Xarray
        paramvec = Yarray

        self.data = data
        self.paramvec = paramvec

    def __len__(self):
        # Number of data point we have. Alternatively self.data.shape[0], or self.label.shape[0]
        return self.size

    def __getitem__(self, idx):
        # Return the idx-th data point of the dataset
        # If we have multiple things to return (data point and label), we can return them as tuple
        data_point = self.data[idx]
        data_paramvec = self.paramvec[idx]
        return data_point, data_paramvec



In [ ]:
batch_size=16

In [ ]:
# Data handling/loading for regression
# For classification, see other notebook


trdat = RVAEDataset(torch.from_numpy(Xtr).to(dtype=torch.float32), torch.from_numpy(Ytr).to(dtype=torch.float32))
trloader = DataLoader(trdat, batch_size=batch_size, shuffle=True) #, shuffle=True

valdat = RVAEDataset(torch.from_numpy(Xval).to(dtype=torch.float32), torch.from_numpy(Yval).to(dtype=torch.float32))
valloader = DataLoader(valdat, batch_size=batch_size, shuffle=True)

tedat = RVAEDataset(torch.from_numpy(Xte).to(dtype=torch.float32), torch.from_numpy(Yte).to(dtype=torch.float32))
teloader = DataLoader(tedat, batch_size=batch_size, shuffle=True)

In [ ]:
enc = nn.Sequential(
                nn.Conv3d(1, outer_layer_dim, kernel_size=5, padding=2),   
                nn.ReLU(),
                nn.Conv3d(outer_layer_dim, 32, kernel_size=5, padding=0), 
                nn.BatchNorm3d(32),
                nn.ReLU(),
                # ResidualBlock(32, 32, kernel_size=3, padding=1),
                nn.MaxPool3d(2),
                nn.Conv3d(32, 32, kernel_size=3, padding=0), 
                nn.BatchNorm3d(32),
                nn.ReLU(),
                nn.MaxPool3d(2),
                nn.Conv3d(32, 64, kernel_size=4, padding=0),
                nn.ReLU(),
                # ResidualBlock(64, 64, kernel_size=3, padding=1),
                nn.Conv3d(64, 128, kernel_size=4, padding=0),
                nn.BatchNorm3d(128),
                nn.ReLU(),
                nn.MaxPool3d(2),
                nn.Conv3d(128, 256, kernel_size=3, padding=1),
                nn.BatchNorm3d(256),
                nn.Conv3d(256, 512, kernel_size=3, padding=0),
                nn.BatchNorm3d(512),
                nn.ReLU(),
                nn.AvgPool3d(2),
                nn.Flatten(start_dim=1, end_dim=-1),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU()
        )

In [ ]:
# encoder = nn.Sequential(
#             nn.Conv3d(1, 512, kernel_size=5, padding=0),
#             nn.ReLU(),

#             nn.MaxPool3d(kernel_size=2),
            
#             nn.Conv3d(512, enc_filt_bl*2, kernel_size=5, padding=0),

#             nn.ReLU(),
    
#             nn.MaxPool3d(kernel_size=2),
    
#             nn.Conv3d(enc_filt_bl*2, enc_filt_bl*2, kernel_size=5, padding=0),

#             nn.ReLU(),
            
            
#             nn.Flatten(start_dim=1, end_dim=-1),
    
#             nn.Linear(enc_filt_bl*2*10*10*10, enc_dim*4),
#             nn.ReLU(),
#             # nn.Dropout(p=0.25),
#             nn.Linear(enc_dim*4, enc_dim*3),
#             nn.ReLU()
#         )

# encoder.to(device)
# torchsummary.summary(encoder, input_size=(1,68,68,68))

In [ ]:
# decoder = nn.Sequential(
#             nn.Linear(enc_dim, enc_dim*4),
#             nn.ReLU(),
#             nn.Linear(enc_dim*4, enc_dim*8),
#             nn.ReLU(),
#             nn.Linear(enc_dim*8, dec_filt_bl*2*10*10*10),
#             nn.ReLU(),
#             nn.Unflatten(1, (dec_filt_bl*2, 10,10,10)),
            
            
#             nn.ConvTranspose3d(dec_filt_bl*2, dec_filt_bl*2, kernel_size=5, padding=0),
#             # nn.BatchNorm3d(num_features = dec_filt_bl*4),
#             nn.ReLU(),
#             nn.Upsample(scale_factor=2, mode='trilinear'),   


#             nn.ConvTranspose3d(dec_filt_bl*2, 512, kernel_size=5, padding=0),
#             # nn.BatchNorm3d(num_features = dec_filt_bl),
#             nn.ReLU(),
    
#             nn.Upsample(scale_factor=2, mode='trilinear'), 
    
#             nn.ConvTranspose3d(512, 1, kernel_size=5, padding=0), 
            
#             nn.ReLU() # nn.ReLU(), # Removing, trying MAE loss... blurry could be sharpened...
#             # nn.Sigmoid()
#         )

# decoder.to(device)
# torchsummary.summary(decoder, input_size=(enc_dim,))

In [ ]:
class SamplingLayer(nn.Module):
    def forward(self, mean, log_var):
        epsilon = torch.randn_like(log_var) # samples are of len(log_var)
        return mean + torch.exp(0.5 * log_var) * epsilon

In [ ]:
class RVAE(nn.Module):
    # def __init__(self, input_shape, latent_dim, ft_bank_baseline, dropout_alpha):
    def __init__(self, input_shape, enc_dim, enc_filt_bl, dec_filt_bl, outer_layer_dim):
    
        super(RVAE, self).__init__()
        # self.latent_dim = latent_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, outer_layer_dim, kernel_size=5, padding=0), 
            nn.ReLU(),

            nn.MaxPool3d(kernel_size=2),
            
            nn.Conv3d(outer_layer_dim, enc_filt_bl*2, kernel_size=5, padding=0),

            nn.ReLU(),
    
            nn.MaxPool3d(kernel_size=2),
    
            nn.Conv3d(enc_filt_bl*2, enc_filt_bl*2, kernel_size=5, padding=0),

            nn.ReLU(),
            
            
            nn.Flatten(start_dim=1, end_dim=-1),
    
            nn.Linear(enc_filt_bl*2*9*9*9, enc_dim*4),
            nn.ReLU(),
            nn.Linear(enc_dim*4, enc_dim*3),
            nn.ReLU()
        )
        

        self.z_mean = nn.Linear(enc_dim*3, enc_dim)
        self.z_log_var = nn.Linear(enc_dim*3, enc_dim)
        
        self.param_mean =  nn.Sequential(nn.Linear(enc_dim*3, enc_dim), 
                                     nn.ReLU(),
                                     nn.Linear(enc_dim, len(params))
                                    )
                                     
        self.param_log_var = nn.Sequential(nn.Linear(enc_dim*3, enc_dim),
                                       nn.ReLU(),
                                       nn.Linear(enc_dim, len(params))
                                      )
                                     
        
        self.sampling = SamplingLayer()
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(enc_dim+len(params), enc_dim*4),
            nn.ReLU(),
            nn.Linear(enc_dim*4, enc_dim*8),
            nn.ReLU(),
            nn.Linear(enc_dim*8, dec_filt_bl*2*9*9*9),
            nn.ReLU(),
            nn.Unflatten(1, (dec_filt_bl*2, 9,9,9)),
            
            
            nn.ConvTranspose3d(dec_filt_bl*2, dec_filt_bl*2, kernel_size=5, padding=0),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='trilinear'),   


            nn.ConvTranspose3d(dec_filt_bl*2, outer_layer_dim, kernel_size=5, padding=0),
            nn.ReLU(),
    
            nn.Upsample(scale_factor=2, mode='trilinear'), 
    
            nn.ConvTranspose3d(outer_layer_dim, 1, kernel_size=5, padding=0), 
            
            # nn.ReLU() # nn.ReLU(), # Removing, trying MAE loss... blurry could be sharpened...
            nn.Sigmoid()
        )
        # self.apply(self._init_weights)  # Apply custom weight initialization to decoder layers
    
    def forward(self, x):
        mean_log_var = self.encoder(x)
        z_mean = self.z_mean(mean_log_var)
        z_log_var = self.z_log_var(mean_log_var)
                                       
        z = self.sampling(z_mean, z_log_var)
        
        param_mean = self.param_mean(mean_log_var)
        param_log_var = self.param_log_var(mean_log_var)
        r = self.sampling(param_mean, param_log_var)
        
        zr = torch.cat((z,r), dim=1)
        
        # reconstructed = self.decoder(z)
        reconstructed = self.decoder(zr)
        
        
        return reconstructed, z_mean, z_log_var, r, param_mean, param_log_var

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv3d, nn.ConvTranspose3d, nn.Linear)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


In [ ]:
# From ChatGPT
# Define your training and validation functions
def rvae_train(net, dataloader, optimizer, beta):
    net.train()  # Set the model to training mode
    running_loss = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    
    for voxel, paramvec in pbar:
        voxel = voxel.to('cuda')  # Move inputs to GPU
        paramvec = paramvec.to('cuda')
        optimizer.zero_grad()
        
        #  # Use the beta_scheduler within the training loop
        # beta = beta_scheduler(epoch, pbar.n, len(dataloader),annealing_type='cosine')
        # beta_history.append(beta)  # Record beta values for plotting
        
        reconstructed, z_mean, z_log_var, r, r_mean, r_log_var = net(voxel)
        loss = RVAE_Loss_wG(reconstructed, voxel, z_mean, z_log_var, paramvec, r, r_mean, r_log_var, beta=beta)
        loss[0].backward()
        optimizer.step()
        running_loss += loss[0].item()
        pbar.set_description(f'Train Loss: {running_loss / (pbar.n + 1):.4f}')
    return running_loss / len(dataloader), loss[1], loss[2], loss[3]


def rvae_validate(net, dataloader, beta):
    net.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    with torch.no_grad():
        for voxel, paramvec in pbar:
            voxel = voxel.to('cuda')  # Move inputs to GPU
            paramvec = paramvec.to('cuda')
            reconstructed, z_mean, z_log_var, r, r_mean, r_log_var = net(voxel)
            loss = RVAE_Loss_wG(reconstructed, voxel, z_mean, z_log_var, paramvec, r, r_mean, r_log_var, beta=beta)
            running_loss += loss[0].item()
            pbar.set_description(f'Val Loss: {running_loss / (pbar.n + 1):.4f}')
    return running_loss / len(dataloader), loss[1], loss[2], loss[3]

In [ ]:
# cycle_length = 200
# plateau_start = 50
# plateau_interval = 20

# # Define a function to schedule the beta value with different annealing types
# def beta_scheduler(epoch, batch_idx, num_batches, annealing_type='linear', 
#                    plateau_start=plateau_start, plateau_interval=plateau_interval, cycle_length=cycle_length):
#     if epoch < plateau_start or (epoch - plateau_start) % plateau_interval != 0:
#         # Before plateau or during non-plateau intervals, apply the selected annealing type
#         if annealing_type == 'linear':
#             beta = min(1.0, epoch / plateau_start)
#         elif annealing_type == 'logistic':
#             k = 10.0
#             beta = 1.0 / (1.0 + torch.exp(-k * (epoch - plateau_start) / plateau_start))
#         elif annealing_type == 'cosine':
#             beta = 0.5 * (1.0 + torch.cos(torch.tensor((epoch % cycle_length) * 2.0 * 3.14159 / cycle_length)))
#         else:
#             raise ValueError("Invalid annealing_type. Choose from 'linear', 'logistic', or 'cosine'.")
#     else:
#         # During plateaus, set beta to the maximum value
#         beta = 1.0

#     return beta

In [ ]:
# # modified from https://github.com/haofuml/cyclical_annealing/blob/master/plot/plot_schedules.ipynb
# def frange_cycle_linear(start, stop, n_epoch, low_plat_len, low_plat_value, n_cycle=4, ratio=0.5):
#     L = np.ones(n_epoch)
#     # for i in range
#     period = n_epoch/n_cycle
#     step = (stop-start)/(period*ratio) # linear schedule

#     for c in range(n_cycle):

#         v , i = start , 0
        
#         while i <= low_plat_len:
#             L[int(i+c*period)] = low_plat_value
#             i += 1
#         while v <= stop and (int(i+c*period) < n_epoch):
#             L[int(i+c*period)] = v
#             v += step
#             i += 1
#     return L    

# def frange_cycle_sigmoid(start, stop, n_epoch, low_plat_len, low_plat_value, n_cycle=4, ratio=0.5):
#     L = np.ones(n_epoch)
#     period = n_epoch/n_cycle
#     step = (stop-start)/(period*ratio) # step is in [0,1]
    
#     # transform into [-6, 6] for plots: v*12.-6.

#     for c in range(n_cycle):

#         v , i = start , 0
#         while i <= low_plat_len:
#             L[int(i+c*period)] = low_plat_value
#             i += 1
#         while v <= stop:
#             L[int(i+c*period)] = 1.0/(1.0+ np.exp(- (v*12.-6.)))
#             v += step
#             i += 1
#     return L    


# #  function  = 1 − cos(a), where a scans from 0 to pi/2

# def frange_cycle_cosine(start, stop, n_epoch, low_plat_len, low_plat_value, n_cycle=4, ratio=0.5):
#     L = np.ones(n_epoch)
#     period = n_epoch/n_cycle
#     step = (stop-start)/(period*ratio) # step is in [0,1]
    
#     # transform into [0, pi] for plots: 

#     for c in range(n_cycle):

#         v , i = start , 0
#         while i <= low_plat_len:
#             L[int(i+c*period)] = low_plat_value
#             i += 1
#         while v <= stop:
#             L[int(i+c*period)] = 0.5-.5*math.cos(v*math.pi)
#             v += step
#             i += 1
#     return L    

# # def frange(start, stop, step, n_epoch):
# #     L = np.ones(n_epoch)
# #     v , i = start , 0
# #     while v <= stop:
# #         L[i] = v
# #         v += step
# #         i += 1
# #     return L


In [ ]:
# # This modified BCE loss function is from https://arxiv.org/pdf/1608.04236.pdf
# # Use of gamma penalizes false negatives while reducing penalty for false positives
# usegamma=True
# class BCEpenalized(nn.Module):
#     def __init__(self,gamma=0.97, usegamma=usegamma):
#         super(BCEpenalized, self).__init__()
#         self.gamma = gamma
        
#     def forward(self, output, target, *args):
#         if usegamma:
#             loss = -self.gamma * target * torch.log(output) - (1 - self.gamma) * (1 - target) * torch.log(1 - output)
#         else:
#             loss = - target * torch.log(output) - (1 - target) * torch.log(1 - output)
            
#         return loss.mean(), None, None
     

In [ ]:
# def Regression_Loss(r, r_mean, r_log_var, param_vector):
#     num_elements = r_mean.size(0)  # Get the number of elements in the tensors
#     regression_loss = 0.5 * torch.pow(r_mean - param_vector, 2) / torch.exp(r_log_var) + 0.5 * r_log_var
#     regression_loss = torch.sum(regression_loss) / num_elements 
#     return regression_loss

In [ ]:
alpha = 1.0
beta = 0.0001
def RVAE_Loss_wG(reconstructed, input_voxel_array, z_mean, z_log_var, param_vector, r, r_mean, r_log_var, beta=0.0001, alpha=1.0):
    
    # reconstruction_loss = BCEpenalized()(reconstructed, input_voxel_array)
    reconstruction_loss = nn.BCELoss()(reconstructed, input_voxel_array)
    
    # reconstruction_loss = nn.MSELoss(reduction='mean')(reconstructed, input_voxel_array)
    
    # regression_loss = Regression_Loss(r, r_mean, r_log_var, param_vector)
    regression_loss = nn.MSELoss()(r, param_vector)
    
    
    kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp())
    
    # loss_value =  reconstruction_loss + (1-alpha)*kl_loss + beta*regression_loss
    loss_value =  reconstruction_loss + beta*kl_loss + alpha*regression_loss
    
    
    return loss_value, reconstruction_loss, kl_loss, regression_loss

lossfunc_name = 'vae_regression_loss'

### <u>__Model parameters__</u>

In [ ]:
# Create the VAE model
rvae = torch.nn.DataParallel(RVAE(input_shape, enc_dim, enc_filt_bl, dec_filt_bl, outer_layer_dim).to(device))


In [ ]:
from torchsummary import summary

In [ ]:
rvae

In [ ]:
summary(rvae.module.param_mean, input_size=(48,))

In [ ]:
summary(rvae.module.encoder, input_size=(1,64,64,64))

In [ ]:
summary(rvae.module.decoder, input_size=(17,))

## <u>__OPTIMIZER HERE__ </u>

In [ ]:
lrate = 0.0001

lratestr = f'{lrate:.0e}'

In [ ]:
# Define the optimizer
optimizer = optim.Adam(rvae.parameters(), lr=lrate)


In [ ]:
cp_name = f'Model_weights_{archver}.pth'

best_weights_path = os.path.join(cp_dir, cp_name)

In [ ]:
print(best_weights_path)

In [ ]:
# Training loop with model weight saving
EPOCHS= 500
patience = 100

min_val_loss = float('inf')
best_val_loss = float('inf')
early_stop_counter = 0
# earlystop_min_delta = 0.000075
earlystop_min_delta = 0.001 # was .00075, but loss for VAE has been around 5e-2 or so, so the magnitude of change should be only one order of magnitude away. Or something. # For L1Loss (MAE)

best_epoch = 0

train_losses = []
val_losses = []


In [ ]:
best_weights_path

In [ ]:
# rvae.load_state_dict(torch.load(best_weights_path))

## Beta schedule for KLD Annealing

In [ ]:
# beta_schedule = frange_cycle_cosine(0.0, 1.0, 300, low_plat_len=20, low_plat_value=0.0001, n_cycle=5, ratio=0.5)

In [ ]:
# plt.plot(range(300), beta_schedule, '-', label='Cyclical', marker= 's', color='k', markevery=None ,lw=2,  mec='k', mew=1 , markersize=10)

In [ ]:
# loss_criterion = vae_loss_mod
# bcepen = BCEpenalized()
# loss_criterion = bcepen

epochs_completed=0

kl_start_epoch = 0

beta = 0.01

try:                
    for epoch in range(EPOCHS):
        
        
        if epoch >= kl_start_epoch:
            # loss_criterion = vae_loss_mod
            # Train the model
            train_loss_output = rvae_train(rvae, trloader, optimizer, beta=beta) #beta_schedule[epoch]
            train_loss = train_loss_output[0]
            reconst_loss = train_loss_output[1]
            kl_loss = train_loss_output[2]
            reg_loss = train_loss_output[3]
            

            # Validate the model
            val_loss_output = rvae_validate(rvae, valloader, beta=beta) # beta_schedule[epoch]
            val_loss = val_loss_output[0]
            vreconst_loss = val_loss_output[1]
            vkl_loss = val_loss_output[2]
            vreg_loss = val_loss_output[3]

            # print("train_loss_output:", train_loss_output)
            # print("val_loss_output:", val_loss_output)
            print(f'train reconst loss: {reconst_loss:.5f}, KL loss: {kl_loss:.5f}, regr loss: {reg_loss:.5f}')
            print(f'  val reconst loss: {vreconst_loss:.5f}, KL loss: {vkl_loss:.5f}, regr loss: {vreg_loss:.5f}')
            
        else:
            train_loss_output = train(rvae, trloader, loss_criterion, optimizer)
            train_loss = train_loss_output[0]
            # reconst_loss = train_loss_output[1]
            # kl_loss = train_loss_output[2]


            # Validate the model
            
            val_loss_output = validate(rvae, valloader, loss_criterion)
            val_loss = val_loss_output[0]
            # vreconst_loss = val_loss_output[1]
            # vkl_loss = val_loss_output[2]

            # print('train_loss:', train_loss_output, train_loss)
            print(f'train reconst loss: {train_loss:.5f}')
            print(f'  val reconst loss: {val_loss:.5f}')
            

        
        # Save the model's weights if validation loss is improved
        improvement_delta = best_val_loss - val_loss
        
        if val_loss < best_val_loss:
            pct_improved = (best_val_loss - val_loss) / best_val_loss * 100
            print(f"Val loss improved from {best_val_loss:.5f} to {val_loss:.5f} ({pct_improved:.2f}% improvement) saving model state...")
            best_val_loss = val_loss
            torch.save(rvae.state_dict(), best_weights_path)  # Save model weights to file
            best_epoch = epoch
        else:
            print(f'Val loss did not improve from {best_val_loss:.5f}.')
            # early_stop_counter += 1  # Increment early stopping counter
            
        if improvement_delta > earlystop_min_delta:
            early_stop_counter = 0
        else:
            early_stop_counter +=1
            
        # Collect model training history
        train_losses.append(train_loss)
        val_losses.append(val_loss)
            
        # Check for early stopping
        if early_stop_counter >= patience:
            print(f'Validation loss did not improve for {early_stop_counter} epochs. Early stopping...')
            rvae.load_state_dict(torch.load(best_weights_path))
            print(f"Model best weights restored - training epoch {best_epoch}")
            break
        
        print(f'Epoch [{epoch+1}/{EPOCHS}]\tTrain Loss: {train_loss:.5f}\tValidation Loss: {val_loss:.5f}')
        
        epochs_completed +=1


    # Load the best weights at end of training epochs
    rvae.load_state_dict(torch.load(best_weights_path))  # Load best model weights
    print(f'Training epochs completed, best model weights restored - epoch {best_epoch}')
    min_val_loss = best_val_loss

except KeyboardInterrupt:
    hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'vall_loss {lossfunc_name}': val_losses}
    rvae.load_state_dict(torch.load(best_weights_path))

In [ ]:
# Run all above...

In [ ]:
hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'vall_loss {lossfunc_name}': val_losses}

In [ ]:
# epochs_completed = 19
histno = 1

In [ ]:
histpath = f'{nbpath}/model_jsons/{archver}_hist{histno}_{epochs_completed}ep.json'
print(histpath)

In [ ]:
with open(f'{histpath}', 'w') as f:
        json.dump(hist_dict, f)

In [ ]:
rvae.load_state_dict(torch.load(best_weights_path))

# Messing with the trained model

In [ ]:
"""
How output works:

    def forward(self, x):
        mean_log_var = self.encoder(x)
        z_mean = self.z_mean(mean_log_var)
        z_log_var = self.z_log_var(mean_log_var)
                                       
        z = self.sampling(z_mean, z_log_var)
        
        r_mean = self.r_mean(mean_log_var)
        r_log_var = self.r_log_var(mean_log_var)
        r = self.sampling(r_mean, r_log_var)
        
        zr = torch.cat((z,r), dim=1)
        
        # reconstructed = self.decoder(z)
        reconstructed = self.decoder(zr)
        
        
*****   return reconstructed, z_mean, z_log_var, r, r_mean, r_log_var   ******


"""

In [ ]:
rvae.eval()

In [ ]:
Xte60_pred_out = rvae(torch.from_numpy(np.expand_dims(Xte[60], axis=0)).to(dtype=torch.float32).to(device))

In [ ]:
Xte144_pred_out = rvae(torch.from_numpy(np.expand_dims(Xte[144], axis=0)).to(dtype=torch.float32).to(device))

In [ ]:
# PlotCell(Xte[60,0,:,:,:]) #intCrossX 	0.27
# PlotCell(Xte[144,0,:,:,:]) #outline_faceCrossX_intRhombX 	0.08

In [ ]:
decoder = rvae.module.decoder

In [ ]:
vec = torch.randn(1,17)

In [ ]:
vec

In [ ]:
example_vec_for_volfrac = tensor([[ 1.0427,  2.4776,  1.1436, -1.1580,  0.0983, -0.0051, -2.0001, -1.4030,
          1.6013, -1.2122, -0.8949, -1.3882, -0.3895,  1.4816, -1.3332, -0.7524,
          0.1500]])

In [ ]:
decoded_array = np.squeeze(decoder(vec.cuda()).detach().cpu().numpy(), axis=(0,1))

In [ ]:



# if convert_to_numpy:
#     modvec = modvec.numpy()
#     decoded_array = decoded_array.numpy()
#     array_thresholded = array_thresholded.numpy()

volfrac = (np.sum(array_thresholded == 1)) / (64**3)

volfrac_str = f'vf{volfrac:.3f}'
volfrac_nodec = f'0p{volfrac_str[-3:]}'

print(f"Volume Fraction: {volfrac_str}")

In [ ]:
voxdic = {}

In [ ]:
# volfracs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
volfracs = [0.15, 0.3, 0.45, 0.6, 0.75, 1.0]

threshold=0.75

for vf in volfracs:
    vf_entry = 'VF'+str(vf*100)+'%'
    voxdic[vf_entry] = {}
    vec[0, 16] = vf
    dec_array = np.squeeze(decoder(vec.cuda()).detach().cpu().numpy(), axis=(0,1))
    
    voxdic[vf_entry]['decoded array'] = dec_array
    
    thresh_array = (dec_array >= threshold).astype(int)
    
    voxdic[vf_entry]['thresholded array'] = thresh_array
    
    volfrac = (np.sum(thresh_array == 1)) / (64**3)
    
    volfrac_pct = str(volfrac * 100)+'%'
    
    volfrac_str = f'{volfrac:.2f}'
    # volfrac_nodec = f'0p{volfrac_str[-3:]}'
    
    voxdic[vf_entry]['volume fraction pct'] = volfrac_pct
    voxdic[vf_entry]['volume fraction str'] = volfrac_str
    
    
    

In [ ]:
for key in voxdic.keys():
    print(key, voxdic[key]['volume fraction pct'])

In [ ]:
# def plot_continuous_array(array, colorscale='sunset', width=800, height=800, marker_scaler=8):
#     z,x,y = array.nonzero()
#     value_list = []
#     predf = pd.DataFrame({'x':x, 'y':y, 'z':z})
#     plotto = int(len(predf)/2)
#     for i in range(len(x)):
#         value_list.append(array[z[i], x[i], y[i]]) # this needs to look up the 3d array loc
#     predf["activation_value"] = value_list
#     predf['markersize_scaler'] = (predf["activation_value"] - predf["activation_value"].min()) / (predf["activation_value"].max() - predf["activation_value"].min())
    
#     p = px.scatter_3d(x=predf['x'][:],  #plotto
#                     y=predf['y'][:], 
#                     z=predf['z'][:], 
#                     color=predf['activation_value'][:], 
#                     opacity=1.0,
#                     color_continuous_scale= "{0}".format(colorscale),
#                     width=width,
#                     height=height)
                    
#     p.update_traces(marker=dict(size=12,
#                                 line=dict(width=0.1,
#                                         color='grey')),)
#     p.update_traces(marker_size = predf['markersize_scaler']*marker_scaler)
#     # p.update_layout(margin=margins)
#     p.show()


In [ ]:
import pandas as pd
import plotly.express as px

def plot_continuous_array_range(array, colorscale='sunset', width=800, height=800, marker_scaler=8, x_range=None,
                                y_range=None, z_range=None):
    z, x, y = array.nonzero()
    
    # Filter data based on specified ranges
    if x_range is not None:
        x_mask = (x >= x_range[0]) & (x <= x_range[1])
        x = x[x_mask]
        y = y[x_mask]
        z = z[x_mask]
        
    if y_range is not None:
        y_mask = (y >= y_range[0]) & (y <= y_range[1])
        x = x[y_mask]
        y = y[y_mask]
        z = z[y_mask]
        
    if z_range is not None:
        z_mask = (z >= z_range[0]) & (z <= z_range[1])
        x = x[z_mask]
        y = y[z_mask]
        z = z[z_mask]

    value_list = []
    predf = pd.DataFrame({'x': x, 'y': y, 'z': z})
    
    for i in range(len(x)):
        value_list.append(array[z[i], x[i], y[i]])
    
    predf["activation_value"] = value_list
    predf['markersize_scaler'] = (predf["activation_value"] - predf["activation_value"].min()) / (predf["activation_value"].max() - predf["activation_value"].min())
    
    p = px.scatter_3d(x=predf['x'], 
                      y=predf['y'], 
                      z=predf['z'], 
                      color=predf['activation_value'], 
                      opacity=1.0,
                      color_continuous_scale=colorscale,
                      width=width,
                      height=height)
                    
    p.update_traces(marker=dict(size=12, line=dict(width=0.1, color='grey')))
    p.update_traces(marker_size=predf['markersize_scaler'] * marker_scaler)
    
    p.show()

# Example usage:
# plot_continuous_array(array, x_range=(0, array.shape[1] // 2), y_range=None, z_range=None)


In [ ]:
voxdic.keys()

In [ ]:
# plot_continuous_array_range(voxdic['VF40.0%']['decoded array'], x_range=(0,32), y_range=(0,32))

In [ ]:
# for key in voxdic.keys():
#     # plot_continuous_array(voxdic[key]['decoded array'])
#     plot_continuous_array_ramge(voxdic[key]['decoded array'], x_range=(0,32))
    
#     # PlotCell(voxdic[key]['thresholded array'])